In [1]:
import math
import numpy as np

def sum_of_lambd(_lambd, _K):
    sum = 0
    for i in range(3, _K+1):
        sum += _lambd ** i
    return sum

def cost_func(_rho, _gamma, _sigma, _mu1, _mu2, _p, _K):
    a0 = ((_rho / _sigma) ** 2) + (_mu1 + _mu2) * _rho / _sigma + _mu1 * _mu2
    a1 = -2 * ((_rho / _sigma) ** 2) - (_mu1 + _mu2) * _rho / _sigma
    a2 = (_rho / _sigma) ** 2
    b0 = (1 - _p) * _mu1 * (_rho / _sigma) + _mu1 * _mu2
    b1 = -((_rho / _sigma) ** 2) - ((2 - _p) * _mu1 + _mu2) * (_rho / _sigma) - _mu1 * _mu2
    b2 = 2 * ((_rho / _sigma) ** 2) + (_mu1 + _mu2) * (_rho / _sigma)
    b3 = -((_rho / _sigma) ** 2)
    alpha2 = a0 / b0
    alpha3 = a1 / b0 - a0 * b1 / (b0 ** 2)
    alpha4 = a2 / b0 - a1 * b1 / (b0 ** 2) + a0 * (b1 ** 2) / (b0 ** 3) - (a0 * b2) / (b0 ** 2)
    lambd1 = (-(b0 + b1) + math.sqrt((b0 + b1)**2 + 4 * b0 * b3)) / (2 * b0)
    lambd2 = (-(b0 + b1) - math.sqrt((b0 + b1)**2 + 4 * b0 * b3)) / (2 * b0)
    d1 = (lambd2 * (alpha3 - alpha2) - (alpha4 - alpha3)) / ((lambd1 ** 3) * (lambd2 - lambd1))
    d2 = (alpha4 - alpha3 - lambd1 * (alpha3 - alpha2)) / ((lambd2 ** 3) * (lambd2 - lambd1))
    alpha_K = alpha2 + d1 * sum_of_lambd(lambd1, _K) + d2 * sum_of_lambd(lambd2, _K)
    return 1 + ((_gamma - alpha_K) / (1 + _rho * alpha_K))

def bisection(_a, _b, gamma_, mu1_, mu2_, p_, K_, sigma_):
    eps = 0.001
    while(abs(_b - _a) > eps):
        x = (_a + _b) / 2
        C1 = cost_func(x - eps, gamma_, sigma_, mu1_, mu2_, p_, K_)
        C2 = cost_func(x + eps, gamma_, sigma_, mu1_, mu2_, p_, K_)
        if (C1 < C2):
            _b = x
        else:
            _a = x
    return x

def get_golden_search(_a, _b, _gamma, _K):
    epsilon = 0.001
    phi = 0.5 * (1.0 + math.sqrt(5.0))
    x1 = _b - ((_b - _a) / phi)
    x2 = _a + ((_b - _a) / phi)
    if(x1 != 1.0):
        y1 = ((1.0 - x1) * (_gamma + x1 ** _K) / (1.0 - x1 ** (_K + 1.0)))
    else:
        y1 = (_gamma + 1.0) / (1.0 + _K)
    if(x2 != 1.0):
        y2 = ((1.0 - x2) * (_gamma + x2 ** _K) / (1.0 - x2 ** (_K + 1.0)))
    else:
        y2 = (_gamma + 1.0) / (1.0 + _K)
    if(y1 >= y2):
        _a = x1
    else:
        _b = x2
    if(abs(_b - _a) < epsilon):
        return (_a + _b) / 2
    else:
        return get_golden_search(_a, _b, _gamma, _K)
    
def get_optimal_rho(_gamma, _K):
    if(_gamma > 0 and _gamma < 1.0):
        a = (_gamma / _K) ** (1.0 / (_K - 1.0))
        b = 1.0
        return get_golden_search(a, b, _gamma, _K)
    elif(_gamma > 1.0):
        a = 1.0
        b = ((_K + 1.0) * _gamma) ** (1.0 / (_K - 1))
        return get_golden_search(a, b, _gamma, _K)
    else:
        return 1.0
    
def get_C(gamma, K, rho):
    p0 = (1 - rho) / (1.0 - rho ** (K + 1))
    pk = (1 - rho) / (1.0 - rho ** (K + 1)) * rho ** K
    return gamma * p0 + pk

def get_optimal_C_N(lambd, mu, gamma, K, rho):
    p0 = 0
    pk = 0
    if(rho == 1.0):
        p0 = 1.0 / (K + 1.0)
        pk = p0
    elif(rho > 0):
        p0 = (1 - rho) / (1.0 - rho ** (K + 1))
        pk = (1 - rho) / (1.0 - rho ** (K + 1)) * rho ** K
    for i in range(0, 19):
        rho = get_optimal_rho(gamma, K)
        n1 = math.floor(rho * mu / lambd)
        n2 = math.ceil(rho * mu / lambd)
        C1 = 0
        C2 = 0
        if((n1 * lambd / mu) != 1.0):
            C1 = (1.0 - (n1 * lambd / mu)) * (gamma + (n1 * lambd / mu) ** K) / (1.0 - (n1 * lambd / mu) ** (K + 1.0))
        else:
            C1 = (gamma + 1.0) / (1.0 + K)
        if((n2 * lambd / mu) != 1.0):
            C2 = (1.0 - (n2 * lambd / mu)) * (gamma + (n2 * lambd / mu) ** K) / (1.0 - (n2 * lambd / mu) ** (K + 1.0))
        else:
            C2 = (gamma + 1.0) / (1.0 + K)
        if(C1 < C2):
            return n1, C1
        else:
            return n2, C2